In [1]:
!pip install https://github.com/explosion/spacy-models/releases/download/en_core_web_lg-2.2.0/en_core_web_lg-2.2.0.tar.gz

     |████████████████████████████████| 827.9 MB 4.3 kB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.0-py3-none-any.whl size=829187827 sha256=4732152785f5dcfedad27b81ff2db929e04b179b1ba0c771ce0e695adf3f4900
  Stored in directory: /root/.cache/pip/wheels/4b/6e/87/8b129988ba4d66c064415e09db36d5945b638aabcaf8f41765
Successfully built en-core-web-lg


In [1]:
!pip install bert-tensorflow

     |████████████████████████████████| 64 kB 2.2 MB/s 


In [4]:
#%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
!ln -s /content/gdrive/MyDrive/ /mydrive
!ls /mydrive

 2021-online-language-offline-impact-main
 An_Xception_Based_Convolutional_Neural_Network_for_Scene_Image_Classification_with_Transfer_Learning.pdf
'Application for Lease- Form for Email (Applicant) (1).gdoc'
'Application for Lease- Form for Email (Applicant).gdoc'
'Application for Lease- Form for Email (Applicant).pdf'
'Colab Notebooks'
'CV -Xizhi Wu.pdf'
'FOW Documents'
 MyDrive
'NT15 LEASE 2021.gdoc'
'NT15 LEASE 2021.pdf'
 Xizhi_Wu_UnderGrad_Transcript.pdf


In [6]:
%cd gdrive/MyDrive/2021-online-language-offline-impact-main

/content/gdrive/MyDrive/2021-online-language-offline-impact-main


In [7]:
!ls

 a0529.csv   a0605.csv	        Heat_Map_Online_language.ipynb
 a0530.csv   csv	        __MACOSX
 a0531.csv   csv.zip	        non_violant1.csv
 a0601.csv   dataframe_csv      non_violant2.csv
 a0602.csv   Github	        violant.csv
 a0603.csv   heatmap530.ipynb  '新建 Microsoft PowerPoint 演示文稿.pptx'
 a0604.csv   heatmap531.ipynb


In [8]:
import os
import collections
import ast
import pandas as pd
import numpy as np
from scipy import stats
from datetime import datetime, timedelta
import spacy
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers
from bert import tokenization
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import matplotlib.pyplot as plt

In [9]:
ct_path_606 = 'csv/US06_06.csv'

In [10]:
ct = pd.read_csv(ct_path_606)

In [11]:
ct.shape[1]

47

In [12]:
columns1=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message','expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media_type', 'media_numl', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']
columns2=['platformId', 'platform', 'date', 'updated', 'type1', 'title', 'caption', 'description', 'message', 'expandedLinks', 'link', 'postUrl', 'subscriberCount', 'score', 'media', 'actual_likeCount', 'actual_shareCount', 'actual_commentCount', 'actual_loveCount', 'actual_wowCount', 'actual_hahaCount', 'actual_sadCount', 'actual_angryCount', 'actual_thankfulCount', 'actual_careCount', 'expected_likeCount', 'expected_shareCount', 'expected_commentCount', 'expected_loveCount', 'expected_wowCount', 'expected_hahaCount', 'expected_sadCount', 'expected_angryCount', 'expected_thankfulCount', 'expected_careCount', 'account_id', 'account_name', 'account_handle', 'account_profileImage', 'account_accountType', 'account_pageAdminTopCountry', 'account_pageDescription', 'account_verified', 'account_pageCreatedDate', 'languageCode', 'legacyId', 'post_id']

In [13]:
#len(columns1)
len(columns2)

47

In [14]:
ct.columns=columns2

In [15]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id
0,19729044948_10151472828499949,Facebook,2020-06-06 20:16:26,2021-07-28 15:31:24,photo,NaN,NaN,"Yesterdays webinar Gender-Justice was, hopeful...",Protestors gather along 16th Street NW near th...,1,https://www.facebook.com/933thebeat/photos/a.3...,https://www.facebook.com/19729044948/posts/101...,56103,-6.000000,1.0,3,1,0,0,0,0,0,0,0,0,3,3,2,3,2,4,7,0,0,6,38774,93.3 The Beat Jamz,933thebeat,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Jacksonville's #1 Station for Hip-Hop and R&B ...,True,2009-01-13 16:46:07,en,0,38774|10151472828499949
1,193425634038780_2897594533621863,Facebook,2020-06-06 20:16:26,2021-06-15 21:27:28,link,George Floyd GoFundMe Campaign Has Received Mo...,incrediblestories.me,A GoFundMe page set up for George Floyd has re...,George Floyd GoFundMe Campaign Has Received Mo...,1,https://incrediblestories.me/george-floyd-gofu...,https://www.facebook.com/193425634038780/posts...,528349,3.857143,1.0,457,100,270,258,20,2,0,0,0,51,146,24,39,54,7,8,5,4,0,8,369803,Terry Bаms,TerryBams,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Army Vet | Best Selling Author | Father | Husb...,True,2011-06-14 05:05:53,en,0,369803|2897594533621863
2,52150999700_10158798739704701,Facebook,2020-06-06 20:16:22,2021-09-02 05:24:41,link,Watch 'Sesame Street' and CNN's town hall on r...,ew.com,'Sesame Street' and CNN presented a one-hour s...,Sesame Street and CNN presented a one-hour spe...,1,http://share.ew.com/X8cbJCW,https://www.facebook.com/52150999700/posts/101...,4609669,1.461864,1.0,628,174,32,188,3,6,1,3,0,36,477,35,52,81,15,19,23,6,0,11,15356,Entertainment Weekly,entertainmentweekly,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Entertainment Weekly brings the fun. With our...,True,2009-02-23 20:21:13,en,0,15356|10158798739704701
3,49727872493_10159055015087494,Facebook,2020-06-06 20:16:03,2021-09-03 11:28:55,link,Protesters pour into DC for city's largest dem...,fox5dc.com,Protesters pour into DC for city's largest dem...,"Here's the scene in downtown DC, where up to 2...",1,https://www.fox5dc.com/news/protesters-pour-in...,https://www.facebook.com/49727872493/posts/101...,1230384,6.021978,1.0,360,108,421,139,32,11,4,21,0,16,45,27,44,7,16,9,18,16,0,5,48870,Fox 5 DC,fox5dc,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Welcome to the official Facebook page for FOX ...,True,2008-12-16 17:46:26,en,0,48870|10159055015087494
4,50614167032_10158759715837033,Facebook,2020-06-06 20:15:57,2021-08-31 04:12:00,link,Outspoken Hoke County Sheriff Sounds Off on Ra...,spectrumlocalnews.com,Peterkin talks about what needs to be done.,Spectrum News anchor Tim Boyum chats with Hoke...,2,http://specne.ws/oHwpnh,https://www.facebook.com/50614167032/posts/101...,134561,-3.666667,1.0,4,1,0,4,0,0,0,0,0,0,5,3,4,3,2,3,7,6,0,3,400156,Spectrum News 1 North Carolina,SpectrumNews1NC,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Spectrum News 1 is a 24-hour local news channe...,True,2008-10-03 00:36:16,en,0,400156|10158759715837033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11360,57427307078_10159378232012079,Facebook

In [16]:
ct['message'] = ct['message'].fillna('')
ct['description'] = ct['description'].fillna('')
ct['title'] = ct['title'].fillna('')

In [17]:
nlp = spacy.load('en_core_web_lg', disable=["tok2vec", "tagger", "parser", "attribute_ruler", "lemmatizer"])

In [18]:
def ner(txt):
    ner = dict()

    doc = nlp(txt)
    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [19]:
def ner_inline(doc):
    ner = dict()

    for ent in doc.ents:
        if ent.label_ == 'GPE':
            if ent.text not in ner:
                ner[ent.text] = 1
            else:
                ner[ent.text] += 1
    return ner

In [20]:
def ner_pipe(texts):
    batch = []
    for doc in nlp.pipe(texts, batch_size=200, n_threads=16):
        batch.append(ner_inline(doc))
    return batch

In [21]:
ct['NER-msg'] = ner_pipe(ct['message'])

In [22]:
ct['NER-msg'] = ct['message'].apply(ner)

In [23]:
ct['NER-desc'] = ct['description'].apply(ner)

In [24]:
ct['NER-title'] = ct['title'].apply(ner)

In [25]:
ct[['NER-msg', 'NER-desc']].head()

,NER-msg,NER-desc
0,"{'Washington': 1, 'D.C.': 1, 'Minneapolis': 1}",{}
1,{},{}
2,{},{}
3,{'DC': 1},"{'DC': 1, 'WASHINGTON': 1}"
4,"{'Hoke County': 1, 'Raeford': 1}",{}


In [26]:
ct

,platformId,platform,date,updated,type1,title,caption,description,message,expandedLinks,link,postUrl,subscriberCount,score,media,actual_likeCount,actual_shareCount,actual_commentCount,actual_loveCount,actual_wowCount,actual_hahaCount,actual_sadCount,actual_angryCount,actual_thankfulCount,actual_careCount,expected_likeCount,expected_shareCount,expected_commentCount,expected_loveCount,expected_wowCount,expected_hahaCount,expected_sadCount,expected_angryCount,expected_thankfulCount,expected_careCount,account_id,account_name,account_handle,account_profileImage,account_accountType,account_pageAdminTopCountry,account_pageDescription,account_verified,account_pageCreatedDate,languageCode,legacyId,post_id,NER-msg,NER-desc,NER-title
0,19729044948_10151472828499949,Facebook,2020-06-06 20:16:26,2021-07-28 15:31:24,photo,,NaN,"Yesterdays webinar Gender-Justice was, hopeful...",Protestors gather along 16th Street NW near th...,1,https://www.facebook.com/933thebeat/photos/a.3...,https://www.facebook.com/19729044948/posts/101...,56103,-6.000000,1.0,3,1,0,0,0,0,0,0,0,0,3,3,2,3,2,4,7,0,0,6,38774,93.3 The Beat Jamz,933thebeat,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Jacksonville's #1 Station for Hip-Hop and R&B ...,True,2009-01-13 16:46:07,en,0,38774|10151472828499949,"{'Washington': 1, 'D.C.': 1, 'Minneapolis': 1}",{},{}
1,193425634038780_2897594533621863,Facebook,2020-06-06 20:16:26,2021-06-15 21:27:28,link,George Floyd GoFundMe Campaign Has Received Mo...,incrediblestories.me,A GoFundMe page set up for George Floyd has re...,George Floyd GoFundMe Campaign Has Received Mo...,1,https://incrediblestories.me/george-floyd-gofu...,https://www.facebook.com/193425634038780/posts...,528349,3.857143,1.0,457,100,270,258,20,2,0,0,0,51,146,24,39,54,7,8,5,4,0,8,369803,Terry Bаms,TerryBams,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Army Vet | Best Selling Author | Father | Husb...,True,2011-06-14 05:05:53,en,0,369803|2897594533621863,{},{},{}
2,52150999700_10158798739704701,Facebook,2020-06-06 20:16:22,2021-09-02 05:24:41,link,Watch 'Sesame Street' and CNN's town hall on r...,ew.com,'Sesame Street' and CNN presented a one-hour s...,Sesame Street and CNN presented a one-hour spe...,1,http://share.ew.com/X8cbJCW,https://www.facebook.com/52150999700/posts/101...,4609669,1.461864,1.0,628,174,32,188,3,6,1,3,0,36,477,35,52,81,15,19,23,6,0,11,15356,Entertainment Weekly,entertainmentweekly,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Entertainment Weekly brings the fun. With our...,True,2009-02-23 20:21:13,en,0,15356|10158798739704701,{},{},{}
3,49727872493_10159055015087494,Facebook,2020-06-06 20:16:03,2021-09-03 11:28:55,link,Protesters pour into DC for city's largest dem...,fox5dc.com,Protesters pour into DC for city's largest dem...,"Here's the scene in downtown DC, where up to 2...",1,https://www.fox5dc.com/news/protesters-pour-in...,https://www.facebook.com/49727872493/posts/101...,1230384,6.021978,1.0,360,108,421,139,32,11,4,21,0,16,45,27,44,7,16,9,18,16,0,5,48870,Fox 5 DC,fox5dc,https://scontent-sea1-1.xx.fbcdn.net/v/t1.1816...,facebook_page,US,Welcome to the official Facebook page for FOX ...,True,2008-12-16 17:46:26,en,0,48870|10159055015087494,{'DC': 1},"{'DC': 1, 'WASHINGTON': 1}",{'DC': 1}
4,50614167032_10158759715837033,Facebook,2020-06-06 20:15:57,2021-08-31 04:12:00,link,Outspoken Hoke County Sheriff Sounds Off on Ra...,spectrumlocalnews.com,Peterkin talks about what needs to be done.,Spectrum News anchor Tim Boyum chats with Hoke...,2,http://specne.ws/oHwpnh,https://www.facebook.com/50614167032/posts/101...,134561,-3.666667,1.0,4,1,0,4,0,0,0,0,0,0,5,3,4,3,2,3,7,6,0,3,400156,Spectrum News 1 North Carolina,SpectrumNews1NC,https://scontent-sea1-1.xx.fbcdn.net/v/t1.6435...,facebook_page,US,Spectrum News 1 is a 24-hour local news channe...,True,2008-10-03 00:36:16,en,0,400156|10158759715837033,"{'Hoke County': 1, 'Raeford': 1}",{},{}
...,...,...,...,...,...,...,...,...,...,...,...,.

In [27]:
ct.to_csv('a0606.csv')

In [28]:
ct=pd.read_csv('a0606.csv',lineterminator='\n')

In [29]:
ct['NER-msg'] = ct['NER-msg'].apply(ast.literal_eval)
ct['NER-desc'] = ct['NER-desc'].apply(ast.literal_eval)
ct['NER-title'] = ct['NER-title'].apply(ast.literal_eval)

In [30]:
ct_ner = ct[((ct['NER-msg'] != {}) | (ct['NER-desc'] != {}) | (ct['NER-title'] != {}))]

In [31]:
ct_ner[['NER-msg', 'NER-desc', 'NER-title']].head()

,NER-msg,NER-desc,NER-title
0,"{'Washington': 1, 'D.C.': 1, 'Minneapolis': 1}",{},{}
3,{'DC': 1},"{'DC': 1, 'WASHINGTON': 1}",{'DC': 1}
4,"{'Hoke County': 1, 'Raeford': 1}",{},{}
5,{'Buffalo': 1},{},{}
8,{'U.S.': 1},{},{}


In [32]:
ct_ner['NER-msg'] = ct_ner['NER-msg'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-desc'] = ct_ner['NER-desc'].apply(lambda x: list(set(x.keys())))
ct_ner['NER-title'] = ct_ner['NER-title'].apply(lambda x: list(set(x.keys())))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [33]:
ct_ner['NER-msg']

0        [Washington, Minneapolis, D.C.]
3                                   [DC]
4                 [Raeford, Hoke County]
5                              [Buffalo]
8                                 [U.S.]
                      ...               
11359                          [Raeford]
11360          [Raeford, North Carolina]
11362                          [Buffalo]
11363                       [Sacramento]
11364           [Monroe County, Florida]
Name: NER-msg, Length: 8109, dtype: object

In [34]:
ct_ner['NER'] = ct_ner['NER-msg'] + ct_ner['NER-desc'] + ct_ner['NER-title']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [35]:
ct_ner['NER']

0                          [Washington, Minneapolis, D.C.]
3                                 [DC, WASHINGTON, DC, DC]
4                                   [Raeford, Hoke County]
5                                                [Buffalo]
8                                                   [U.S.]
                               ...                        
11359                   [Raeford, Raeford, North Carolina]
11360    [Raeford, North Carolina, Raeford, North Carol...
11362                 [Buffalo, Buffalo, BUFFALO, Buffalo]
11363              [Sacramento, Buffalo, BUFFALO, Buffalo]
11364    [Monroe County, Florida, Monroe County, Florid...
Name: NER, Length: 8109, dtype: object

In [36]:
ct_ner['NER'] = ct_ner['NER'].apply(set)
ct_ner['NER'] = ct_ner['NER'].apply(list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [37]:
ct_ner['NER'] = ct_ner['NER'].apply(lambda x: [s.lower() for s in x])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [38]:
ner_lst = ct_ner['NER'].values

In [39]:
ner_lst

array([list(['washington', 'd.c.', 'minneapolis']),
       list(['washington', 'dc']), list(['raeford', 'hoke county']), ...,
       list(['buffalo', 'buffalo']),
       list(['buffalo', 'sacramento', 'buffalo']),
       list(['monroe county', 'florida', 'dui'])], dtype=object)

In [40]:
m=0
list=[]
for n in ner_lst:
  for i in range(len(n)):
    list.append(ner_lst[m][i])
  m+=1

In [41]:
list

['washington',
 'd.c.',
 'minneapolis',
 'washington',
 'dc',
 'raeford',
 'hoke county',
 'buffalo',
 'u.s.',
 'washington',
 'kent',
 'northampton',
 'washington',
 'u.s.',
 'd.c.',
 'bloomfield',
 'minneapolis',
 'n.y.',
 'buffalo',
 'orange county',
 'america',
 'minneapolis',
 'new york city',
 'london',
 'london',
 'london',
 'london',
 'london',
 'london',
 'london',
 'london',
 'london',
 'minneapolis',
 'minneapolis',
 'erie county',
 'amherst',
 'buffalo',
 'u.s.',
 'north carolina',
 'hoke county',
 'nc',
 'hoke county',
 'us',
 'nc',
 'north carolina',
 'oconomowoc',
 '3:00pm',
 'burleigh',
 'shorewood',
 'wi',
 'milwaukee',
 'san jose',
 'houston',
 'houston',
 'missouri',
 'minneapolis',
 'houston',
 'missouri',
 'houston',
 'missouri',
 'houston',
 'missouri',
 'staten island',
 'minneapolis',
 'staten island',
 'minneapolis',
 'america',
 'nj',
 'new jersey',
 'new jersey',
 'minnesota',
 'delray beach city commission',
 'boca',
 'boca raton',
 'delray beach city commis

In [42]:
c = collections.Counter()

In [43]:
dict_a=collections.Counter(list).most_common()

In [44]:
dict_a

[('minneapolis', 2086),
 ('america', 723),
 ('u.s.', 657),
 ('minnesota', 573),
 ('north carolina', 513),
 ('washington', 493),
 ('buffalo', 447),
 ('us', 337),
 ('the united states', 337),
 ('raeford', 239),
 ('dc', 238),
 ('d.c.', 219),
 ('chicago', 206),
 ('maryland', 196),
 ('new york', 178),
 ('california', 164),
 ('houston', 154),
 ('texas', 153),
 ('los angeles', 138),
 ('philadelphia', 136),
 ('new york city', 135),
 ('atlanta', 130),
 ('seattle', 114),
 ('michigan', 109),
 ('nc', 105),
 ('florida', 101),
 ('portland', 96),
 ('sacramento', 91),
 ('london', 88),
 ('nyc', 84),
 ('santa monica', 78),
 ('san francisco', 76),
 ('n.c.', 75),
 ('chicago heights', 74),
 ('hoke county', 72),
 ('n.y.', 67),
 ('australia', 67),
 ('denver', 67),
 ('fayetteville', 64),
 ('new jersey', 62),
 ('nj', 61),
 ('cincinnati', 61),
 ('milwaukee', 58),
 ('paris', 57),
 ('ohio', 57),
 ('ga', 55),
 ('newark', 55),
 ('dallas', 53),
 ('wisconsin', 50),
 ('louisville', 46),
 ('nashville', 45),
 ('hollywoo

In [45]:
dataframe606=pd.DataFrame(dict_a)

In [46]:
len(dict_a)

1923

In [47]:
dataframe606.to_csv('dataframe0606.csv')